# 01: Fit Networks

In [49]:
import os
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

from contextualized.easy import ContextualizedMarkovNetworks as ContextualizedNetworks
from contextualized.baselines.networks import MarkovNetwork as PopulationNetwork
from contextualized.baselines.networks import GroupedNetworks

In [ ]:
# Load or make splits
split_df = pd.read_csv('splits.csv')
train_sample_ids = split_df[split_df['Set'] == 'Train']['sample_id'].values
test_sample_ids = split_df[split_df['Set'] == 'Test']['sample_id'].values

In [ ]:
# Preprocess Data
# Some options
tumor_only = False      # Set to True to exclude healthy samples
disease_labels = True   # Set to False to exclude disease labels from the context for disease hold-out splits
test = True             # Set to False to hold out a portion of the train set instead for development and tuning
dry_run = False         # Set to True to run a small subset of the data for debugging

# Data files to consider
data_dir = "data/"
covariate_files = [
    "clinical_covariates.csv",
    "snv_covariates.csv",
    "scna_other_covariates.csv",
    "scna_arm_covariates.csv",
    "scna_gene_covariates.csv",
]

# load and align data
covars = pd.read_csv(os.path.join(data_dir, covariate_files[0]), header=0)
for covariate_file in covariate_files[1:]:
    covars = covars.merge(
        pd.read_csv(data_dir + covariate_file, header=0),
        on="sample_id",
        how="inner",
    )
genomic = pd.read_csv(os.path.join(data_dir, "transcriptomic_features.csv"), header=0)
covars_genomic = covars.merge(genomic, on="sample_id", how="inner")
covars = covars_genomic[covars.columns]
genomic = covars_genomic[genomic.columns]
tnames_full = np.load(os.path.join(data_dir, "transcript_names.npy"))
tnames_full = np.array(list(set(genomic.columns).intersection(set(tnames_full))))

# Remove uninteresting features
def drop_cols(df):
    drop_list = []
    for col in df.columns:
        if len(df[col].unique()) < 2:  # drop columns with only one value
            drop_list.append(col)
    df = df.drop(columns=drop_list)
    print(f"Dropped singular columns: {drop_list}")
    return df
covars = drop_cols(covars)

# Remove non-cancerous samples, 2 types of samples: Solid Tissue Normal and Primary Tumor
if tumor_only:
    cancer_idx = covars["sample_type"] != "Solid Tissue Normal"
    covars = covars[cancer_idx]
    genomic = genomic[cancer_idx]

# deterministic numeric encoding for contexts of interest
context_df = covars.copy()
# if the datatype are numeric or not
numeric_covars = {
    "age_at_diagnosis": True,
    "gender": False,
    "year_of_birth": True,
    "race": False,
    "sample_type": False,
    "percent_stromal_cells": True,
    "percent_tumor_cells": True,
    "percent_normal_cells": True,
    "percent_neutrophil_infiltration": True,
    "percent_lymphocyte_infiltration": True,
    "percent_monocyte_infiltration": True,
    "Purity": True,
    "Ploidy": True,
    "WGD": False,
    "stage": True,
}

# if disease_labels set to true, then add the following to the numeric_covars
# disease label determine if this is included in the context information
if disease_labels:
    print("Using disease labels")
    numeric_covars.update(
        {
            "primary_site": False,
            "disease_type": False,
        }
    )

# get a list of column names with following keywords
mut_cols = [
    col
    for col in covars.columns
    if "Arm" in col or "Gene" in col or "Allele" in col or "Mutated" in col
]
# update dictionary with the list of column names in mut_cols and the value for these new keys are True
numeric_covars.update({f"{gene}": True for gene in mut_cols})

transform_i = None
col_count = 0
numeric_headers = []
for (
    col,
    numeric,
) in numeric_covars.items():  # iterate through each entry of the dictionary
    if col in mut_cols and transform_i is None:
        transform_i = col_count
    if numeric:
        # Convert numeric values to floats, replace NaN with column mean
        num_col = pd.to_numeric(context_df[col], errors="coerce")
        num_col = num_col.fillna(num_col.mean())  # imputation here
        num_header = col + "_numeric"
        numeric_headers.append(num_header)
        context_df[num_header] = num_col
        col_count += 1
    else:
        # Make one-hot encoded dummy variables for categorical variables
        dummies = pd.get_dummies(context_df[col], prefix=col)
        numeric_headers += dummies.columns.tolist()
        context_df = context_df.merge(dummies, left_index=True, right_index=True)
        col_count += dummies.shape[-1]
context_df = context_df.drop(columns=covars.columns)
context_df = context_df.copy()
print(f"df contains NaN: {context_df.isnull().values.any()}")

# Get dataset values
C = context_df.values.astype(float)
X = genomic.drop(columns="sample_id").values.astype(float)
print(f"Context shape {C.shape}, Expression shape {X.shape}")

# Remove uninteresting genes
consistent_feats = np.mean(X > 0, axis=0) > 0.999
X = X[:, consistent_feats]
tnames_full = tnames_full[consistent_feats]

# Get train-test split
train_idx = np.isin(covars["sample_id"], train_sample_ids)
test_idx = np.isin(covars["sample_id"], test_sample_ids)
labels = covars["disease_type"].values
tcga_ids = covars["sample_id"].values

C_train, C_test = C[train_idx], C[test_idx]
X_train, X_test = X[train_idx], X[test_idx]
labels_train, labels_test = labels[train_idx], labels[test_idx]
tcga_ids_train, tcga_ids_test = tcga_ids[train_idx], tcga_ids[test_idx]
if not test:
    train_idx, val_idx = train_test_split(
        range(len(X_train)), test_size=0.2, random_state=0
    )
    C_train, C_test = C_train[train_idx], C_train[val_idx]
    X_train, X_test = X_train[train_idx], X_train[val_idx]
    labels_train, labels_test = labels_train[train_idx], labels_train[val_idx]
    tcga_ids_train, tcga_ids_test = (
        tcga_ids_train[train_idx],
        tcga_ids_train[val_idx],
    )

# Compress expression data into metagenes
pca = PCA(n_components=50, random_state=1).fit(X_train)
X_train = pca.transform(X_train)
X_test = pca.transform(X_test)
tnames_full = np.array([f"PC{i}" for i in range(X_train.shape[1])])

# Normalize the data
C_means = np.mean(C_train, axis=0)
C_stds = np.std(C_train, axis=0)
C_stds[C_stds == 0] = 1
C_train -= C_means
C_test -= C_means
C_train /= C_stds
C_test /= C_stds

X_means = np.mean(X_train, axis=0)
X_stds = np.std(X_train, axis=0)
X_stds[X_stds == 0] = 1
X_train -= X_means
X_test -= X_means
X_train /= X_stds
X_test /= X_stds

print(f"Covariates: {C_train.shape} {C_test.shape} {len(C)}")
print(f"Features: {X_train.shape} {X_test.shape} {len(X)}")

# dry run for sanity checking
if dry_run:
    n = 100
    C_train[:n],
    C_test[:n],
    X_train[:n],
    X_test[:n],
    labels_train[:n],
    labels_test[:n],
    tcga_ids_train[:n],
    tcga_ids_test[:n],
    tnames_full,

Dropped singular columns: ['EPAS1 Mutated', 'MBD1 Mutated', 'NRAS Mutated', 'PSIP1 Mutated', '13p Major Arm Copies', '14p Major Arm Copies', '15p Major Arm Copies', '22p Major Arm Copies', '13p Minor Arm Copies', '14p Minor Arm Copies', '15p Minor Arm Copies', '22p Minor Arm Copies', '13p Arm Duplicated', '14p Arm Duplicated', '15p Arm Duplicated', '21p Arm Duplicated', '22p Arm Duplicated', '13p Arm Loss of Heterozygosity', '14p Arm Loss of Heterozygosity', '15p Arm Loss of Heterozygosity', '21p Arm Loss of Heterozygosity', '22p Arm Loss of Heterozygosity', '1p Arm Lost', '1q Arm Lost', '2p Arm Lost', '2q Arm Lost', '3p Arm Lost', '3q Arm Lost', '4p Arm Lost', '4q Arm Lost', '5p Arm Lost', '5q Arm Lost', '6p Arm Lost', '6q Arm Lost', '7p Arm Lost', '7q Arm Lost', '8p Arm Lost', '8q Arm Lost', '9p Arm Lost', '9q Arm Lost', '10p Arm Lost', '10q Arm Lost', '11p Arm Lost', '11q Arm Lost', '12p Arm Lost', '12q Arm Lost', '13p Arm Lost', '13q Arm Lost', '14p Arm Lost', '14q Arm Lost', '15p 

/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/1309724862.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  context_df[num_header] = num_col
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/1309724862.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  context_df[num_header] = num_col
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/1309724862.py:107: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has

df contains NaN: False
Context shape (7133, 1674), Expression shape (7133, 685)
Covariates: (5707, 1674) (1426, 1674) 7133
Features: (5707, 50) (1426, 50) 7133


## Fit networks and evaluate

__Note:__ 
Here we only run one bootstrap for each method to save time.
In the study, we run 30 bootstraps of each method to get bootstrap-averaged predictions and confidence intervals for each sample.

In general, bootstrapping is especially helpful for flexible methods like contextualized networks which are more prone to overfitting.
Without bootstrapping, errors may be higher than in the original study.

For tutorials on built-in bootstrapping and other modeling utilities, see the [contextualized.ml documentation](https://contextualized.ml/docs/intro.html).

In [54]:
# Fit a population model
pn = PopulationNetwork().fit(X_train)
print('test MSE ', pn.measure_mses(X_test).mean())

test MSE  0.9720281573306225


In [55]:
# Fit a disease-specific model
disease_networks = GroupedNetworks(PopulationNetwork).fit(X_train, labels_train)
print('test MSE ', disease_networks.measure_mses(X_test, labels_test).mean())

test MSE  0.6172512033149444


In [56]:
# Fit a cluster-specific model
kmeans = KMeans(n_clusters=len(np.unique(labels_train)), random_state=1).fit(C_train)
clusters_train = kmeans.predict(C_train)
clusters_test = kmeans.predict(C_test)
cluster_networks = GroupedNetworks(PopulationNetwork).fit(X_train, clusters_train)
print('test MSE ', cluster_networks.measure_mses(X_test, clusters_test).mean())

test MSE  0.9242100512463282


In [57]:
# Fit a contextualized model
cn = ContextualizedNetworks()
cn.fit(C_train, X_train)
print('test MSE ', cn.measure_mses(C_test, X_test).mean())

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/pnas/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:177: GPU available but not used. You can set it by doing `Trainer(accelerator='gpu')`.
/opt/homebrew/Caskroom/miniforge/base/envs/pnas/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
/opt/homebrew/Caskroom/miniforge/base/envs/pnas/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/calebellington/Documents/Manuscripts/Network-based Molecular Profiles of Cancer/PNAS Revisions/DataReprocessing/CancerContextualized_revisions/lightning_logs/boot_0_checkpoints exists and is not empty.

  | Name      | Type             | Params | Mode 
-------------------------------------------------

test MSE  0.593113739208252


/opt/homebrew/Caskroom/miniforge/base/envs/pnas/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


## Save Networks for Visualization and Subtyping

In [62]:
train_networks = cn.predict_networks(C_train)
test_networks = cn.predict_networks(C_test)

/opt/homebrew/Caskroom/miniforge/base/envs/pnas/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'predict_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.


In [69]:
feature_names = [f"PC{i}-PC{j}" for i in range(X_train.shape[1]) for j in range(X_train.shape[1])]
train_networks_df = pd.DataFrame(tcga_ids_train, columns=['sample_id'])
train_networks_df['Set'] = 'Train'
train_networks_df[feature_names] = train_networks.reshape(len(train_networks), -1)
test_networks_df = pd.DataFrame(tcga_ids_test, columns=['sample_id'])
train_networks_df['Set'] = 'Test'
test_networks_df[feature_names] = test_networks.reshape(len(test_networks), -1)
networks_df = pd.concat([train_networks_df, test_networks_df])
networks_df.to_csv('data/networks.csv', index=False)

/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/2646668219.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_networks_df[feature_names] = train_networks.reshape(len(train_networks), -1)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/2646668219.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  train_networks_df[feature_names] = train_networks.reshape(len(train_networks), -1)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_5285/2646668219.py:4: PerformanceWarning: DataFrame is 